In [1]:
from googleapiclient.discovery import build
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
API_KEY = os.getenv("YOUTUBE_API_KEY")
CHANNEL_ID = "UCCuzDCoI3EUOo_nhCj4noSw"

# Define start and end date
START_DATE = "2025-01-21T00:00:00Z"
END_DATE = "2025-03-29T00:00:00Z"


# Function to fetch data from YouTube API and handle pagination
def fetch_video_data(api_key, channel_id, start_date, end_date):
    video_details = []
    page_token = None  # To store the nextPageToken for pagination

    while True:
        # Construct the search URL
        search_url = f"https://www.googleapis.com/youtube/v3/search?part=snippet&channelId={channel_id}&type=video&eventType=completed&publishedAfter={start_date}&publishedBefore={end_date}&maxResults=50&key={api_key}"

        if page_token:
            search_url += f"&pageToken={page_token}"

        # Make the request to YouTube API
        search_response = requests.get(search_url).json()

        if "items" in search_response:
            video_ids = [item["id"]["videoId"] for item in search_response["items"]]

            # Get video details (statistics like views, likes, etc.)
            stats_url = f"https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics&id={','.join(video_ids)}&key={api_key}"
            stats_response = requests.get(stats_url).json()

            for item in stats_response.get("items", []):
                video_info = {
                    "video_id": item["id"],  # ✅ Adding Video ID
                    "title": item["snippet"]["title"],
                    "video_url": f"https://www.youtube.com/watch?v={item['id']}",
                    "published_date": item["snippet"].get("publishedAt", "N/A"),
                    "views": item["statistics"].get("viewCount", "N/A"),
                    "likes": item["statistics"].get("likeCount", "N/A"),
                    "comments": item["statistics"].get("commentCount", "N/A"),
                }
                video_details.append(video_info)

        # Check if there is another page of results
        page_token = search_response.get("nextPageToken")
        if not page_token:
            break  # No more pages to fetch

    return video_details


# Fetch the video data
video_data = fetch_video_data(API_KEY, CHANNEL_ID, START_DATE, END_DATE)

# Save the data to a CSV file
if video_data:
    df = pd.DataFrame(video_data)
    df.to_csv("youtube_live_streams.csv", index=False)
    print("Data successfully saved to youtube_live_streams.csv")
else:
    print("No archived live streams found within the given date range.")

Data successfully saved to youtube_live_streams.csv
